In [2]:
import sqlite3
con = sqlite3.connect("tutorial.db")

In [3]:
cur = con.cursor()

In [4]:
cur.execute("CREATE TABLE movie(title,year,score)")

In [5]:
res = cur.execute("SELECT name FROM sqlite_master")
res.fetchone()

('movie',)

In [6]:
res = cur.execute("SELECT name FROM sqlite_master WHERE name='spam'")
res.fetchone() is None

True

In [15]:
cur.execute("""INSERT INTO movie VALUES
    ('Monty Python and the Holy Grail',1975,8.2),
    ('And Now for Something Completely Different',1971,7.5)
""")

In [16]:
con.commit()

In [17]:
res = cur.execute("SELECT score FROM movie")

In [18]:
res.fetchall()

[(8.2,), (7.5,)]

In [19]:
data = [
    ("Monty Python Live at the Hollywood Bowl",1982,7.9),
    ("Monty Python's The Meaning of Life",1983,7.5),
    ("Monty Python's Life of Brain",1979,8.0),
]
cur.executemany("INSERT INTO movie VALUES(?,?,?)", data)
con.commit()  # Remember to commit the transaction after executing INSERT.

In [20]:
for row in cur.execute("SELECT year, title FROM movie ORDER BY year"):
    print(row)

(1971, 'And Now for Something Completely Different')
(1975, 'Monty Python and the Holy Grail')
(1979, "Monty Python's Life of Brain")
(1982, 'Monty Python Live at the Hollywood Bowl')
(1983, "Monty Python's The Meaning of Life")


In [21]:
con.close()
new_con = sqlite3.connect("tutorial.db")
new_cur = new_con.cursor()
res = new_cur.execute("SELECT title, year FROM movie ORDER BY score DESC")
title, year = res.fetchone()
print(f'The highest scoring Monty Python movie is {title!r},released in {year}')

The highest scoring Monty Python movie is 'Monty Python and the Holy Grail',released in 1975


In [24]:
sqlite3.complete_statement("SELECT foo FROM bar;")

True

In [25]:
sqlite3.complete_statement("SELECT foo")

False

In [31]:
sqlite3.enable_callback_tracebacks(True)
con = sqlite3.connect(":memory:")
def evil_trace(stmt):
    5/0
con.set_trace_callback(evil_trace)
def debug(unraisable):
    print(f"{unraisable.exc_value!r} in callback {unraisable.object.__name__}")
    print(f"Error message: {unraisable.err_msg}")  
import sys
sys.unraisablehook = debug
cur = con.execute("SELECT 1")     

ZeroDivisionError: division by zero

In [35]:
import hashlib
def md5sum(t):
    return hashlib.md5(t).hexdigest()
con = sqlite3.connect(":memory:")
con.create_function("md5", 1, md5sum)
for row in con.execute("SELECT md5(?)",(b"foo",)):
    print(row)

('acbd18db4cc2f85cedef654fccc4a4d8',)


In [37]:
class MySum:
    def __init__(self):
        self.count = 0
      
    def step(self, value):
        self.count += value
        
    def finalize(self):
        return self.count
    
con = sqlite3.connect(":memory:")
con.create_aggregate("mysum", 1, MySum)
cur = con.execute("CREATE TABLE test(i)")
cur.execute("INSERT INTO test(i) VALUES(1)")
cur.execute("INSERT INTO test(i) VALUES(2)")
cur.execute("SELECT mysum(i) FROM test")
print(cur.fetchone()[0])

con.close()

3


In [39]:
class WindowSumInt:
    def __init__(self):
        self.count = 0

    def step(self, value):
        """Add a row to the current window."""
        self.count += value

    def value(self):
        """Return the current value of the aggregate."""
        return self.count

    def inverse(self, value):
        """Remove a row from the current window."""
        self.count -= value

    def finalize(self):
        """Return the final value of the aggregate.

        Any clean-up actions should be placed here.
        """
        return self.count


con = sqlite3.connect(":memory:")
cur = con.execute("CREATE TABLE test(x, y)")
values = [
    ("a", 4),
    ("b", 5),
    ("c", 3),
    ("d", 8),
    ("e", 1),
]
cur.executemany("INSERT INTO test VALUES(?, ?)", values)
con.create_window_function("sumint", 1, WindowSumInt)
cur.execute("""
    SELECT x, sumint(y) OVER (
        ORDER BY x ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING
    ) AS sum_y
    FROM test ORDER BY x
""")
print(cur.fetchall())

AttributeError: 'sqlite3.Connection' object has no attribute 'create_window_function'

In [40]:
def collate_reverse(string1, string2):
    if string1 == string2:
        return 0
    elif string1 < string2:
        return 1
    else:
        return -1
    
con = sqlite3.connect(":memory:")
con.create_collation("reverse",collate_reverse)

cur = con.execute("CREATE TABLE test(x)")
cur.executemany("INSERT INTO test(x) VALUES(?)",[("a",),("b",)])
cur.execute("SELECT x FROM test ORDER BY x COLLATE reverse")
for row in cur:
    print(row)
    
con.close()    

('b',)
('a',)


In [48]:
con.enable_load_extension(True)

# Load the fulltext search extension
con.execute("select load_extension('./fts3.so')")

# alternatively you can load the extension using an API call:
# con.load_extension("./fts3.so")

# disable extension loading again
con.enable_load_extension(False)

# example from SQLite wiki
con.execute("CREATE VIRTUAL TABLE recipe USING fts3(name,ingredients)")
con.executescript("""
INSERT INTO recipe (name, ingredients) VALUES('broccoli stew',
'broccoli peppers cheese tomatoes');
INSERT INTO recipe (name, ingredients) VALUES('pumpkin stew',
'pumpkin onions garlic celery');
INSERT INTO recipe (name, ingredients) VALUES('broccoli pie',
'broccoli cheese onions flour');
INSERT INTO recipe (name, ingredients) VALUES('pumpkin pie',
'pumpkin sugar flour butter');
""")
for row in con.execute("SELECT rowid, name, ingredients FROM recipe WHERE name MATCH 'pie'"):
    print(row)
con.close()

ProgrammingError: Cannot operate on a closed database.

In [51]:
# Convert file example.db to SQL dump file dump.sql
con = sqlite3.connect('example.db')
with open('dump.sql', 'w') as f:
    for line in con.iterdump():
        f.write('%s\n' % line)
con.close()

In [52]:
def progress(status, remaining, total):
    print(f'Copied {total-remaining} of {total} pages...')
src = sqlite3.connect('example.db')
dst = sqlite3.connect('backup.db')
with dst:
    src.backup(dst, pages=1, progress=progress)
dst.close()
src.close()

Copied 0 of 0 pages...


In [53]:
src = sqlite3.connect('example.db')
dst = sqlite3.connect(':memory:')
src.backup(dst)

In [54]:
con.getlimit(sqlite3.SQLITE_LIMIT_SQL_LENGTH)

AttributeError: 'sqlite3.Connection' object has no attribute 'getlimit'

In [55]:
 con.setlimit(sqlite3.SQLITE_LIMIT_ATTACHED, 1)

AttributeError: 'sqlite3.Connection' object has no attribute 'setlimit'

In [57]:
con = sqlite3.connect(":memory:")
cur = con.cursor()
AUSTRIA = "Österreich"
# by default, rows are returned as str
cur.execute("SELECT ?", (AUSTRIA,))
row = cur.fetchone()
assert row[0] == AUSTRIA
# but we can make sqlite3 always return bytestrings ...
con.text_factory = bytes
cur.execute("SELECT ?", (AUSTRIA,))
row = cur.fetchone()
assert type(row[0]) is bytes
# the bytestrings will be encoded in UTF-8, unless you stored garbage in the
# database ...
assert row[0] == AUSTRIA.encode("utf-8")
# we can also implement a custom text_factory ...
# here we implement one that appends "foo" to all strings
con.text_factory = lambda x: x.decode("utf-8") + "foo"
cur.execute("SELECT ?", ("bar",))
row = cur.fetchone()
assert row[0] == "barfoo"
con.close()

In [62]:
con = sqlite3.connect(":memory:")
rows = [
    ("row1",),
    ("row2",),
]
# cur is an sqlite3.Cursor object
cur.executemany("INSERT INTO data VALUES(?)", rows)

OperationalError: no such table: data

In [63]:
con = sqlite3.connect(":memory:")
cur.executescript("""
BEGIN;
CREATE TABLE person(firstname, lastname, age);
CREATE TABLE book(title, author, published);
CREATE TABLE publisher(name, address);
COMMIT;
""")

In [64]:
con = sqlite3.connect(":memory:")
cur = con.cursor()
cur.connection == con

True

In [66]:
con = sqlite3.connect(":memory:")
con.execute("CREATE TABLE test(blob_col blob)")
con.execute("INSERT INTO test(blob_col) VALUES(zeroblob(13))")
# Write to our blob, using two write operations:
with con.blobopen("test", "blob_col", 1) as blob:
    blob.write(b"hello, ")
    blob.write(b"world.")
    # Modify the first and last bytes of our blob
    blob[0] = ord("H")
    blob[-1] = ord("!")
# Read the contents of our blob
with con.blobopen("test", "blob_col", 1) as blob:
    greeting = blob.read()
print(greeting) # outputs "b'Hello, world!'""

AttributeError: 'sqlite3.Connection' object has no attribute 'blobopen'